In [1]:
import os
import onnx
import torch
from torch import nn
import tensorflow as tf
from onnx_tf.backend import prepare
from torchvision.models import resnet18,mobilenet_v2,ResNet18_Weights
import torchvision
import torchvision.transforms as transforms


data_dir = os.path.abspath("../../data")
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), transforms.RandomErasing()])
trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

dataiter = iter(trainloader) 
images, labels = next(dataiter)

model = resnet18(weights = ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(in_features=512,out_features=10)


onnx_file = "../../results/Coral_Resnet18_model.onnx"
tf_model_path = "../../results/Coral_Resnet18_model.tf"


torch.onnx.export(
    model,
    images,
    onnx_file,
    opset_version=12,
    do_constant_folding=True,
)

onnx_model =  onnx.load(onnx_file)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

# tf_model = tf.keras.models.Sequential([
#     tf.keras.layers.InputLayer(input_shape=(3, 32, 32)),
#     tf.keras.layers.Lambda(lambda x: x.permute(0, 2, 3, 1)),
#     tf.keras.layers.Lambda(lambda x: model(torch.Tensor(x.numpy()).permute(0, 3, 1, 2)).detach().numpy())
# ])

2023-02-28 21:22:11.777501: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 21:22:12.493193: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/slurm/slurm-19.05.5/lib:/usr/local/slurm/slurm-19.05.5/lib/slurm:/usr/local/lib:
2023-02-28 21:22:12.493284: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:

Files already downloaded and verified


2023-02-28 21:22:15.690926: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-28 21:22:15.690980: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (wr0): /proc/driver/nvidia/version does not exist
2023-02-28 21:22:15.691762: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


AttributeError: Exception encountered when calling layer "lambda" (type Lambda).

'Tensor' object has no attribute 'permute'

Call arguments received by layer "lambda" (type Lambda):
  • inputs=tf.Tensor(shape=(None, 3, 32, 32), dtype=float32)
  • mask=None
  • training=None

In [ ]:
tflite_model_path = "../../results/Coral_Resnet18_model.tflite"

IMAGE_SIZE = 32
(train_images, train_labels), (_, _) = tf.keras.datasets.cifar10.load_data()
def representative_data_gen():
    for i in range(100):
        # Get a random image from the CIFAR-10 training dataset
        image = train_images[i % len(train_images)]
        # Resize the image to the desired size
        image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
        # Normalize the pixel values
        image = tf.cast(image / 255., tf.float32)
        # Add an extra dimension to represent a batch of size 1
        image = tf.expand_dims(image, 0)
        # Yield the image as representative data
        yield [image]



converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
#converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.target_spec.supported_types = [tf.int8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8


tflite_model = converter.convert()
with open(tflite_model_path, 'wb') as f: 
    f.write(tflite_model)